In [26]:
import pandas as pd
import numpy as np
import os.path
import math
from keras import Sequential
from keras.layers import Dense, LSTM

In [27]:
# read in data
fiction = pd.read_csv("fiction.csv", delimiter=",")
isbnToInfo = pd.read_csv("isbnToInfo.csv", delimiter=",")
genreData = pd.read_csv("sortedGenresFiction.csv", delimiter=",")
genreData.fillna(0, inplace=True)  # replace na with 0

In [42]:
def getMatrix(given_csv, k):
    sorted = {}

    # format into dictionary
    for i in range(given_csv.shape[0]):  # per day
        for j in range(1, 21):  # per entry
            if given_csv.iloc[i, j] in sorted:  # if key exist
                sorted[given_csv.iloc[i, j]].append([given_csv.iloc[i, 0], j])
            else:  # key doesn't exist
                sorted[given_csv.iloc[i, j]] = [[given_csv.iloc[i, 0], j]]

    # remove ones that have less than k entries since hard to predict
    clean = {}
    weeks = {}
    for i in sorted.keys():
        if len(sorted[i]) >= k:
            clean[i] = []
            weeks[i] = []
            for j in sorted[i]:
                clean[i].append(j[1])
                weeks[i].append(j[0])

    isbns = list(clean.keys())

    print(clean)

    # add first 3
    nX = []
    nY = []
    nList = []
    maeList = []
    goodISBNs = []
    badEntriesIndex = []

    for i in clean.keys():
        nX.append(clean[i][0:3])

    print('nx length = ', len(nX))

    # graphing a specific book
    wannaCheckIsbn = '0525952926'
    wannaCheckIsbnIndex = 0
    wannaCheckIsbnPredictions = []
    wannaCheckIsbnActual = clean[wannaCheckIsbn][0:k]

    # model
    for n in range(3, k):

        params = []
        nList.append(n)
        nY.clear()

        for j in clean.keys():
            nY.append(clean[j][n])
            goodISBNs.append(j)

        for i in range(len(nX)):  # per book

            # ranking
            # slope, intercept, r_value, p_value, std_err = stats.linregress(list(range(len(nX[i]))), nX[i])
            model = np.polyfit(list(range(len(nX[i]))), nX[i], 2)
            last = nX[i][len(nX[i]) - 1]
            slope = model[0]
            slope1 = model[1]

            # genre
            curISBN = isbns[i]

            listI = isbnToInfo[['isbn']].values.tolist()
            flattenedI = [item for sublist in listI for item in sublist]
            curi = flattenedI.index(curISBN)
            curGenre = isbnToInfo[['Category']].iloc[curi][0]

            if "/" in curGenre:  # take first genre if there are 2
                curGenre = curGenre.split("/")[0]
            prevGenrePercent = []
            listG = fiction[['date']].values.tolist()
            flattened = [item for sublist in listG for item in sublist]
            curWeek = flattened.index(weeks[curISBN][0])

            for j in range(curWeek - 3, curWeek):
                prevGenrePercent.append(genreData[[curGenre]].iloc[j][0])

            # gSlope, gIntercept, gR_value, gP_value, gStd_err = stats.linregress(list(range(len(prevGenrePercent))), prevGenrePercent)
            model = np.polyfit(list(range(len(prevGenrePercent))), prevGenrePercent, 2)
            gSlope0 = model[0]
            gSlope1 = model[1]

            # searches
            curISBNnoZero = curISBN[1:]
            if os.path.isfile(
                    "datadump/" + curISBN + ".csv"):  # for now, until we get all the data
                curSearchesDirty = pd.read_csv(
                    "datadump/" + curISBN + ".csv",
                    delimiter=",")  # cuz not clean
            elif os.path.isfile("datadump/" + curISBNnoZero + ".csv"):
                curSearchesDirty = pd.read_csv(
                    "datadump/" + curISBNnoZero + ".csv", delimiter=",")
            else:
                goodISBNs.remove(curISBN)
                # if n > 3:
                #     print('suprise!')
                badEntriesIndex.append(i)
                continue
            curSearchesBusty = curSearchesDirty[['GT_SearchIndex']].values.tolist()  # busty cuz not flat
            curSearches = [item for sublist in curSearchesBusty for item in sublist]  # flatten
            if len(curSearches) == 0:  # if file empty
                badEntriesIndex.append(i)
                continue

            count = 0
            for j in curSearches:
                if j == 0:  # if searches that day is 0
                    count += 1

            if count / len(curSearches) > 0.25:  # if more than 25% of the searches is 0, change to weekly
                curSearchesWeekly = []
                count = 0
                accumalativeSearches = 0
                totalIterations = 0
                for j in curSearches:
                    # for j in curSearches[4:]:

                    totalIterations += 1
                    count += 1
                    accumalativeSearches += j
                    if count == 7:  # every 7 days (1 week), we record the sales that week
                        if totalIterations < 7 * (k + 4):  # cuz start data starts from 1 month before
                            curSearchesWeekly.append(accumalativeSearches)
                            accumalativeSearches = 0
                            count = 0

            elif count / len(curSearches) <= 0.25:
                curSearchesWeekly = []
                totalIterations = 0

                for j in curSearches:
                    # for j in curSearches[4:]:
                    if totalIterations < 7 * (k + 4):
                        totalIterations += 1
                        curSearchesWeekly.append(j)  # well its rly daily searches but too lazy to make another var

            if len(curSearchesWeekly) < k:
                badEntriesIndex.append(i)


    print(len(nX))

    #####################################
    #####################################
    #####################################
    #####################################
    #####################################
    #####################################

    matrix = []
    bullshitassbullshit = []
    sorted = {}

    # format into dictionary
    for i in range(given_csv.shape[0]):  # per day
        for j in range(1, 21):  # per entry
            if given_csv.iloc[i, j] in sorted:  # if key exist
                sorted[given_csv.iloc[i, j]].append([given_csv.iloc[i, 0], j])
            else:  # key doesn't exist
                sorted[given_csv.iloc[i, j]] = [[given_csv.iloc[i, 0], j]]

    # remove ones that have less than k entries since hard to predict
    clean = {}
    weeks = {}
    for i in sorted.keys():
        if len(sorted[i]) >= k:
            clean[i] = []
            weeks[i] = []
            for j in sorted[i]:
                clean[i].append(j[1])
                weeks[i].append(j[0])

    isbns = list(clean.keys())

    # add first 3
    nX = []
    nY = []
    nList = []
    maeList = []
    # goodISBNs = []
    badEntriesIndex = []

    for i in clean.keys():
        nX.append(clean[i][0:k])


    xlength = len(clean.keys())
    ylength = k
    zlength = 3

    for a in range(xlength):  # num of books

        if a in badEntriesIndex:
            continue

        tempArray2 = []
        for b in range(ylength):  # num of weeks
            tempArray = []

            for c in range(3):
                tempArray.append(0)
            tempArray2.append(tempArray)
        matrix.append(tempArray2)

    matriy = []
    for a in range(xlength):  # num of books
        # matriy.append([None]*3)
        matriy.append([None]*1)

        # matriy.append([])
    print(np.shape(matrix))
    print(np.shape(matriy))

    variable = 0
    skipNextTurn = False
    numOfBadCount = 0
    for a in range(xlength):  # num of books
    # while variable < range(xlength):

        if a in badEntriesIndex:
            # variable = variable-1
            numOfBadCount += 1
            continue

        #####################################
        # genre
        curISBN = isbns[a-numOfBadCount]

        listI = isbnToInfo[['isbn']].values.tolist()
        flattenedI = [item for sublist in listI for item in sublist]
        curi = flattenedI.index(curISBN)
        curGenre = isbnToInfo[['Category']].iloc[curi][0]

        if "/" in curGenre:  # take first genre if there are 2
            curGenre = curGenre.split("/")[0]
        listG = fiction[['date']].values.tolist()
        flattened = [item for sublist in listG for item in sublist]
        curWeek = flattened.index(weeks[curISBN][0])

        # searches
        curISBNnoZero = curISBN[1:]
        if os.path.isfile(
                "datadump/" + curISBN + ".csv"):  # for now, until we get all the data
            curSearchesDirty = pd.read_csv(
                "datadump/" + curISBN + ".csv",
                delimiter=",")  # cuz not clean
        elif os.path.isfile("datadump/" + curISBNnoZero + ".csv"):
            curSearchesDirty = pd.read_csv(
                "datadump/" + curISBNnoZero + ".csv", delimiter=",")
        # else:
        #     # goodISBNs.remove(curISBN)
        #     badEntriesIndex.append(a)
        #     continue
        curSearchesBusty = curSearchesDirty[['GT_SearchIndex']].values.tolist()  # busty cuz not flat
        curSearches = [item for sublist in curSearchesBusty for item in sublist]  # flatten
        # if len(curSearches) == 0:  # if file empty
        #     badEntriesIndex.append(a)
        #     continue

        count = 0
        for j in curSearches:
            if j == 0:  # if searches that day is 0
                count += 1

        # if count / len(curSearches) > 0.25 and len(curSearches)/7 >= k:  # if more than 25% of the searches is 0, change to weekly
        # if True:
        #     curSearchesWeekly = []
        #     count = 0
        #     accumalativeSearches = 0
        #     totalIterations = 0
        #     howMany = math.floor(len(curSearches) / k)
        #     for j in curSearches[4:(7*k + 4)]:
        #         # for j in curSearches[4:]:
        #
        #         totalIterations += 1
        #         count += 1
        #         accumalativeSearches += j
        #         if count == howMany:  # every 7 days (1 week), we record the sales that week
        #             # if totalIterations < 7 * (k + 4):  # cuz start data starts from 1 month before
        #             if True:
        #                 curSearchesWeekly.append(accumalativeSearches)
        #                 accumalativeSearches = 0
        #                 count = 0
        curSearchesWeekly = []
        for j in range(k):
            howMany = math.floor(len(curSearches)/k)
            cumalative = 0
            for l in curSearches[(j*howMany): (j*(howMany+1))]:
                cumalative += l
            curSearchesWeekly.append(cumalative)

        # elif count / len(curSearches) <= 0.25:
        #     curSearchesWeekly = []
        #     totalIterations = 0
        #
        #     for j in curSearches[4:]:
        #         # for j in curSearches[4:]:
        #         if totalIterations < 7 * (k + 4):
        #             totalIterations += 1
        #             curSearchesWeekly.append(j)  # well its rly daily searches but too lazy to make another var


        #####################################

        for b in range(ylength):  # num of weeks
            # past, genre, google

            # print(matrix)
            if b < k-1:
                matrix[a-numOfBadCount][b][0] = nX[a-numOfBadCount][b]
                matrix[a-numOfBadCount][b][1] = genreData[[curGenre]].iloc[curWeek][0]
                try:
                    # print(b, curSearchesWeekly)
                    matrix[a-numOfBadCount][b][2] = curSearchesWeekly[b]
                except:
                    # del matrix[a][b]
                    # print(b, len(curSearchesWeekly))
                    bullshitassbullshit.append([a, b])
            else:
                # matriy[a][0] = nX[a][b]
                # matriy[a][1] = genreData[[curGenre]].iloc[curWeek][0]
                # matriy[a][2] = curSearchesWeekly[b]
                # print(nX[a])
                # print(nX[a][b])
                # matriy[a][0] = nX[a]
                numOfBadCount2 = 0
                for l in range(len(nX)):
                    if l in badEntriesIndex:
                        numOfBadCount2 += 1
                    if l not in badEntriesIndex:
                        try:
                            matriy[l-numOfBadCount2] = nX[l][k-1]
                        except:
                            print(nX[l])
                            print(nX[l][k])
                            print(matriy[l - numOfBadCount2])

        # variable += 1
    #print(bullshitassbullshit)
    # for i in reversed(bullshitassbullshit):
    #     print(i[0])
    #     print(i[1])

        # del matrix[i[0]][i[1]]

    # print(matrix)
    # print(np.shape(matrix))
    
    matrix = np.asarray(matrix)
    matriy = np.asarray(matriy)
    print(np.shape(matrix))
    print(np.shape(matriy))
    # print(matrix)
    #print(matriy)
    return matrix, matriy

In [45]:
matrix, matriy = getMatrix(fiction, 10)

{'0316068047': [2, 2, 2, 3, 3, 5, 6, 7, 4, 3, 3, 5, 6, 5, 5, 6, 6, 7, 10, 11, 9, 11, 12, 12, 13, 15, 17, 10, 8, 5, 5, 5, 3, 2, 2, 2, 2, 4, 3, 4, 8, 6, 4, 4, 5, 2, 4, 5, 6, 8, 8, 7, 7, 9, 11, 13, 17, 15, 16, 17, 18, 16, 12, 16, 19, 19, 17, 17], '0312348673': [3, 4, 4, 5, 5, 10, 11, 10, 13, 15, 14, 13, 15, 18], '0061537934': [13, 14, 15, 13, 15, 17, 17, 17, 17, 19, 20], '0743290119': [6, 6, 6, 4, 8, 13, 12, 10, 10, 10, 11, 18], '0061374229': [14, 9, 6, 5, 4, 2, 2, 4, 6, 7, 8, 6, 9, 9, 1, 1, 2, 2, 3, 5, 6, 9, 11, 10, 7, 4, 4, 4, 4, 6, 5, 5, 3, 3, 6, 7, 8, 13, 16, 20], '0385340990': [5, 3, 4, 3, 3, 5, 3, 5, 9, 12, 15, 16, 15, 13, 16, 17, 13, 18, 18, 16, 15, 19, 17, 10, 8, 8, 9, 8, 8, 6, 10, 10, 11, 7, 9, 13, 13, 9, 14, 15], '0307269752': [4, 7, 10, 12, 13, 17, 17, 20, 19, 17, 18, 20], '0446579939': [1, 1, 2, 3, 4, 6, 9, 11, 9, 12, 8, 7, 7, 10, 16, 16, 17], '0743270428': [1, 2, 5, 7, 12, 16, 16, 17, 18, 17], '0446533424': [1, 4, 6, 8, 12, 15, 14, 13, 13, 16, 19], '0446195502': [1, 3, 5, 8, 

150
(150, 10, 3)
(150, 1)
(150, 10, 3)
(150,)


In [46]:
print(matrix)

[[[2.00e+00 5.00e-02 0.00e+00]
  [2.00e+00 5.00e-02 1.50e+01]
  [2.00e+00 5.00e-02 4.50e+01]
  ...
  [7.00e+00 5.00e-02 3.38e+02]
  [4.00e+00 5.00e-02 3.45e+02]
  [0.00e+00 0.00e+00 0.00e+00]]

 [[3.00e+00 3.00e-01 0.00e+00]
  [4.00e+00 3.00e-01 0.00e+00]
  [4.00e+00 3.00e-01 0.00e+00]
  ...
  [1.00e+01 3.00e-01 0.00e+00]
  [1.30e+01 3.00e-01 0.00e+00]
  [0.00e+00 0.00e+00 0.00e+00]]

 [[1.30e+01 3.00e-01 0.00e+00]
  [1.40e+01 3.00e-01 0.00e+00]
  [1.50e+01 3.00e-01 0.00e+00]
  ...
  [1.70e+01 3.00e-01 0.00e+00]
  [1.70e+01 3.00e-01 0.00e+00]
  [0.00e+00 0.00e+00 0.00e+00]]

 ...

 [[4.00e+00 4.00e-01 0.00e+00]
  [1.00e+01 4.00e-01 6.30e+01]
  [7.00e+00 4.00e-01 6.30e+01]
  ...
  [1.00e+01 4.00e-01 2.84e+02]
  [1.20e+01 4.00e-01 2.42e+02]
  [0.00e+00 0.00e+00 0.00e+00]]

 [[2.00e+00 4.00e-01 0.00e+00]
  [4.00e+00 4.00e-01 0.00e+00]
  [5.00e+00 4.00e-01 1.33e+02]
  ...
  [1.30e+01 4.00e-01 1.99e+02]
  [1.30e+01 4.00e-01 2.84e+02]
  [0.00e+00 0.00e+00 0.00e+00]]

 [[1.00e+00 4.00e-01 0.0

In [38]:
model = Sequential()
model.add(LSTM(units=30, return_sequences=True, input_shape=(matrix.shape[1], 3)))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(units=1))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 10, 30)            4080      
_________________________________________________________________
lstm_5 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
lstm_6 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 18,751
Trainable params: 18,751
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(matrix, matriy, epochs=400, batch_size=32)

Epoch 1/400
150/150 [==============================] - 2s 13ms/step - loss: 192.9796 - acc: 0.0000e+00
Epoch 2/400
150/150 [==============================] - 0s 399us/step - loss: 181.8989 - acc: 0.0400
Epoch 3/400
150/150 [==============================] - 0s 406us/step - loss: 165.5235 - acc: 0.0267
Epoch 4/400
150/150 [==============================] - 0s 426us/step - loss: 143.1865 - acc: 0.0200
Epoch 5/400
150/150 [==============================] - 0s 419us/step - loss: 120.6276 - acc: 0.0400
Epoch 6/400
150/150 [==============================] - 0s 412us/step - loss: 101.9716 - acc: 0.0133
Epoch 7/400
150/150 [==============================] - 0s 432us/step - loss: 87.7824 - acc: 0.0200
Epoch 8/400
150/150 [==============================] - 0s 372us/step - loss: 75.6631 - acc: 0.0600
Epoch 9/400
150/150 [==============================] - 0s 479us/step - loss: 65.2933 - acc: 0.0400
Epoch 10/400
150/150 [==============================] - 0s 419us/step - loss: 58.5915 - acc: 0.0133


150/150 [==============================] - 0s 479us/step - loss: 30.5363 - acc: 0.0333
Epoch 83/400
150/150 [==============================] - 0s 426us/step - loss: 30.5325 - acc: 0.0333
Epoch 84/400
150/150 [==============================] - 0s 412us/step - loss: 30.5319 - acc: 0.0333
Epoch 85/400
150/150 [==============================] - 0s 445us/step - loss: 30.5301 - acc: 0.0333
Epoch 86/400
150/150 [==============================] - 0s 472us/step - loss: 30.5317 - acc: 0.0333
Epoch 87/400
150/150 [==============================] - 0s 472us/step - loss: 30.5285 - acc: 0.0333
Epoch 88/400
150/150 [==============================] - 0s 459us/step - loss: 30.5293 - acc: 0.0333
Epoch 89/400
150/150 [==============================] - 0s 465us/step - loss: 30.5284 - acc: 0.0333
Epoch 90/400
150/150 [==============================] - 0s 426us/step - loss: 30.5293 - acc: 0.0333
Epoch 91/400
150/150 [==============================] - 0s 426us/step - loss: 30.5372 - acc: 0.0333
Epoch 92/400


150/150 [==============================] - 0s 392us/step - loss: 30.5245 - acc: 0.0333
Epoch 164/400
150/150 [==============================] - 0s 485us/step - loss: 30.5307 - acc: 0.0333
Epoch 165/400
150/150 [==============================] - 0s 412us/step - loss: 30.5311 - acc: 0.0333
Epoch 166/400
150/150 [==============================] - 0s 392us/step - loss: 30.5226 - acc: 0.0333
Epoch 167/400
150/150 [==============================] - 0s 406us/step - loss: 30.5211 - acc: 0.0333
Epoch 168/400
150/150 [==============================] - 0s 426us/step - loss: 30.5197 - acc: 0.0333
Epoch 169/400
150/150 [==============================] - 0s 452us/step - loss: 30.5236 - acc: 0.0333
Epoch 170/400
150/150 [==============================] - 0s 465us/step - loss: 30.5201 - acc: 0.0333
Epoch 171/400
150/150 [==============================] - 0s 426us/step - loss: 30.5205 - acc: 0.0333
Epoch 172/400
150/150 [==============================] - 0s 439us/step - loss: 30.5233 - acc: 0.0333
Epoc

150/150 [==============================] - 0s 445us/step - loss: 4.0671 - acc: 0.1733
Epoch 245/400
150/150 [==============================] - 0s 452us/step - loss: 4.0810 - acc: 0.2133
Epoch 246/400
150/150 [==============================] - 0s 445us/step - loss: 3.9484 - acc: 0.1533
Epoch 247/400
150/150 [==============================] - 0s 426us/step - loss: 3.9865 - acc: 0.1800
Epoch 248/400
150/150 [==============================] - 0s 399us/step - loss: 3.8774 - acc: 0.1533
Epoch 249/400
150/150 [==============================] - 0s 406us/step - loss: 3.9689 - acc: 0.1533
Epoch 250/400
150/150 [==============================] - 0s 419us/step - loss: 3.9666 - acc: 0.1667
Epoch 251/400
150/150 [==============================] - 0s 392us/step - loss: 3.7528 - acc: 0.1933
Epoch 252/400
150/150 [==============================] - 0s 426us/step - loss: 3.7953 - acc: 0.1733
Epoch 253/400
150/150 [==============================] - 0s 439us/step - loss: 3.6225 - acc: 0.2000
Epoch 254/400


150/150 [==============================] - 0s 439us/step - loss: 1.4499 - acc: 0.3933
Epoch 327/400
150/150 [==============================] - 0s 386us/step - loss: 1.4614 - acc: 0.4000
Epoch 328/400
150/150 [==============================] - 0s 459us/step - loss: 1.3885 - acc: 0.4200
Epoch 329/400
150/150 [==============================] - 0s 399us/step - loss: 1.3661 - acc: 0.4333
Epoch 330/400
150/150 [==============================] - 0s 386us/step - loss: 1.4064 - acc: 0.4200
Epoch 331/400
150/150 [==============================] - 0s 379us/step - loss: 1.3460 - acc: 0.4667
Epoch 332/400
150/150 [==============================] - 0s 372us/step - loss: 1.3107 - acc: 0.4867
Epoch 333/400
150/150 [==============================] - 0s 392us/step - loss: 1.2822 - acc: 0.4600
Epoch 334/400
150/150 [==============================] - 0s 399us/step - loss: 1.3515 - acc: 0.4667
Epoch 335/400
150/150 [==============================] - 0s 379us/step - loss: 1.3445 - acc: 0.4333
Epoch 336/400


In [40]:
predmatriy = model.predict(matrix)

In [47]:
matrix

array([[[2.00e+00, 5.00e-02, 0.00e+00],
        [2.00e+00, 5.00e-02, 1.50e+01],
        [2.00e+00, 5.00e-02, 4.50e+01],
        ...,
        [7.00e+00, 5.00e-02, 3.38e+02],
        [4.00e+00, 5.00e-02, 3.45e+02],
        [0.00e+00, 0.00e+00, 0.00e+00]],

       [[3.00e+00, 3.00e-01, 0.00e+00],
        [4.00e+00, 3.00e-01, 0.00e+00],
        [4.00e+00, 3.00e-01, 0.00e+00],
        ...,
        [1.00e+01, 3.00e-01, 0.00e+00],
        [1.30e+01, 3.00e-01, 0.00e+00],
        [0.00e+00, 0.00e+00, 0.00e+00]],

       [[1.30e+01, 3.00e-01, 0.00e+00],
        [1.40e+01, 3.00e-01, 0.00e+00],
        [1.50e+01, 3.00e-01, 0.00e+00],
        ...,
        [1.70e+01, 3.00e-01, 0.00e+00],
        [1.70e+01, 3.00e-01, 0.00e+00],
        [0.00e+00, 0.00e+00, 0.00e+00]],

       ...,

       [[4.00e+00, 4.00e-01, 0.00e+00],
        [1.00e+01, 4.00e-01, 6.30e+01],
        [7.00e+00, 4.00e-01, 6.30e+01],
        ...,
        [1.00e+01, 4.00e-01, 2.84e+02],
        [1.20e+01, 4.00e-01, 2.42e+02],
        [

In [60]:
inputX = []
inputY = []

#initialize
for i in range(np.shape(matrix)[0]):
    inputX.append(matrix[i][0:3][:])
    inputY.append(matrix[i][3][0])
print(inputX)
print(inputY)

[array([[ 2.  ,  0.05,  0.  ],
       [ 2.  ,  0.05, 15.  ],
       [ 2.  ,  0.05, 45.  ]]), array([[3. , 0.3, 0. ],
       [4. , 0.3, 0. ],
       [4. , 0.3, 0. ]]), array([[13. ,  0.3,  0. ],
       [14. ,  0.3,  0. ],
       [15. ,  0.3,  0. ]]), array([[ 6.  ,  0.35,  0.  ],
       [ 6.  ,  0.35,  0.  ],
       [ 6.  ,  0.35, 94.  ]]), array([[14.  ,  0.35,  0.  ],
       [ 9.  ,  0.35,  0.  ],
       [ 6.  ,  0.35, 94.  ]]), array([[ 5. ,  0.3,  0. ],
       [ 3. ,  0.3,  0. ],
       [ 4. ,  0.3, 21. ]]), array([[ 4. ,  0.2,  0. ],
       [ 7. ,  0.2,  0. ],
       [10. ,  0.2,  0. ]]), array([[1.  , 0.45, 0.  ],
       [1.  , 0.45, 0.  ],
       [2.  , 0.45, 0.  ]]), array([[1.  , 0.45, 0.  ],
       [2.  , 0.45, 0.  ],
       [5.  , 0.45, 0.  ]]), array([[ 1.  ,  0.25,  0.  ],
       [ 4.  ,  0.25, 18.  ],
       [ 6.  ,  0.25, 18.  ]]), array([[ 1. ,  0.5,  0. ],
       [ 3. ,  0.5, 55. ],
       [ 5. ,  0.5, 28. ]]), array([[ 1.  ,  0.65,  0.  ],
       [ 2.  ,  0.65, 51.  ],

In [91]:
arr = np.array([1,1,1])
print(arr)
#inputX[0][:][3] = np.array(1,1,1)
#np.concatenate(inputX[0][:], np.array([1,2,3]))
merged = []
for l in range():
    merged.append( list(inputX[0][l]) )
print(np.asarray(merged))

[1 1 1]
[[ 2.    0.05  0.  ]
 [ 2.    0.05 15.  ]
 [ 2.    0.05 45.  ]]


In [101]:
inputX = []
inputY = []

#initialize
for i in range(np.shape(matrix)[0]):
    inputX.append(matrix[i][0:3][:])
    inputY.append(matrix[i][3][0])

for j in range(3, 11): #to predict jth result
    if j!=3:
        for i in range(np.shape(matrix)[0]):
            merged = []
            for l in range(j-1):
                merged.append( list(inputX[0][l]) )
            #print(merged)
            
            merged.append([inputY[i], inputX[i][j-2][1], inputX[i][j-2][2]])
            inputX[i] = np.asarray(merged)
    print(inputX)
    print('FINISHED RANK ' + str(j) + ' PREDICTION')

[array([[ 2.  ,  0.05,  0.  ],
       [ 2.  ,  0.05, 15.  ],
       [ 2.  ,  0.05, 45.  ]]), array([[3. , 0.3, 0. ],
       [4. , 0.3, 0. ],
       [4. , 0.3, 0. ]]), array([[13. ,  0.3,  0. ],
       [14. ,  0.3,  0. ],
       [15. ,  0.3,  0. ]]), array([[ 6.  ,  0.35,  0.  ],
       [ 6.  ,  0.35,  0.  ],
       [ 6.  ,  0.35, 94.  ]]), array([[14.  ,  0.35,  0.  ],
       [ 9.  ,  0.35,  0.  ],
       [ 6.  ,  0.35, 94.  ]]), array([[ 5. ,  0.3,  0. ],
       [ 3. ,  0.3,  0. ],
       [ 4. ,  0.3, 21. ]]), array([[ 4. ,  0.2,  0. ],
       [ 7. ,  0.2,  0. ],
       [10. ,  0.2,  0. ]]), array([[1.  , 0.45, 0.  ],
       [1.  , 0.45, 0.  ],
       [2.  , 0.45, 0.  ]]), array([[1.  , 0.45, 0.  ],
       [2.  , 0.45, 0.  ],
       [5.  , 0.45, 0.  ]]), array([[ 1.  ,  0.25,  0.  ],
       [ 4.  ,  0.25, 18.  ],
       [ 6.  ,  0.25, 18.  ]]), array([[ 1. ,  0.5,  0. ],
       [ 3. ,  0.5, 55. ],
       [ 5. ,  0.5, 28. ]]), array([[ 1.  ,  0.65,  0.  ],
       [ 2.  ,  0.65, 51.  ],

       [ 4.  ,  0.4 , 28.  ]])]
FINISHED 7 RANK PREDICTION
[array([[ 2.  ,  0.05,  0.  ],
       [ 2.  ,  0.05, 15.  ],
       [ 2.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ]]), array([[ 2.  ,  0.05,  0.  ],
       [ 2.  ,  0.05, 15.  ],
       [ 2.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 5.  ,  0.3 ,  0.  ]]), array([[ 2.  ,  0.05,  0.  ],
       [ 2.  ,  0.05, 15.  ],
       [ 2.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [ 3.  ,  0.05, 45.  ],
       [13.  ,  0.3 ,  0.  ]]), array([[2.0e+00, 5.0e-02, 0.0e+00],
       [2.0e+00, 5.0e-02, 1.5e+01],
       [2.0e+00, 5.0e-02, 4.5e+01],
       [3.0e+00, 5.0e-02, 4.5e+01],
       [3.0e+00, 5.0e-02, 4.5e+01],
       [3.0e+00, 5.0e-02, 4.5e+01]

In [104]:
inputX = []
inputY = []

#initialize
for i in range(np.shape(matrix)[0]):
    inputX.append(matrix[i][0:3][:])
    inputY.append(matrix[i][3][0])

for j in range(3, 11): #to predict jth result
    if j!=3:
        for i in range(np.shape(matrix)[0]):
            merged = []
            for l in range(j-1):
                merged.append( list(inputX[0][l]) )
            print(merged)
            
            merged.append([inputY[i], inputX[i][j-2][1], inputX[i][j-2][2]])
            inputX[i] = np.asarray(merged)
            
    inputX = np.asarray(inputX)          
    model = Sequential()
    model.add(LSTM(units=30, return_sequences=True, input_shape=(inputX.shape[1], 3)))
    model.add(LSTM(units=30, return_sequences=True))
    model.add(LSTM(units=30))
    model.add(Dense(units=1))
    print(model.summary())
    
    print('MODEL IS TRAINING TO PREDICT THE ' + str(j) + 'TH RANK')
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    model.fit(inputX, inputY, epochs=200, batch_size=32)
    
    inputY = model.predict(inputX)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 3, 30)             4080      
_________________________________________________________________
lstm_14 (LSTM)               (None, 3, 30)             7320      
_________________________________________________________________
lstm_15 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 18,751
Trainable params: 18,751
Non-trainable params: 0
_________________________________________________________________
None
MODEL IS TRAINING TO PREDICT THE 3TH RANK
Epoch 1/200
150/150 [==============================] - 5s 33ms/step - loss: 50.4537 - acc: 0.0000e+00
Epoch 2/200
150/150 [==============================] - 0s 298us/step - loss: 49.2983 - acc: 0.0000e+00
Epoch 3/200
150/15

150/150 [==============================] - 0s 239us/step - loss: 2.0628 - acc: 0.3200
Epoch 75/200
150/150 [==============================] - 0s 239us/step - loss: 1.9992 - acc: 0.3600
Epoch 76/200
150/150 [==============================] - 0s 253us/step - loss: 1.9500 - acc: 0.3267
Epoch 77/200
150/150 [==============================] - 0s 266us/step - loss: 1.9567 - acc: 0.3133
Epoch 78/200
150/150 [==============================] - 0s 193us/step - loss: 1.9645 - acc: 0.3667
Epoch 79/200
150/150 [==============================] - 0s 232us/step - loss: 1.9934 - acc: 0.2733
Epoch 80/200
150/150 [==============================] - 0s 199us/step - loss: 1.9952 - acc: 0.3267
Epoch 81/200
150/150 [==============================] - 0s 223us/step - loss: 1.8469 - acc: 0.3133
Epoch 82/200
150/150 [==============================] - 0s 226us/step - loss: 1.8131 - acc: 0.3333
Epoch 83/200
150/150 [==============================] - 0s 217us/step - loss: 1.7832 - acc: 0.3400
Epoch 84/200
150/150 [=

150/150 [==============================] - 0s 219us/step - loss: 1.1019 - acc: 0.3733
Epoch 157/200
150/150 [==============================] - 0s 220us/step - loss: 1.0972 - acc: 0.3733
Epoch 158/200
150/150 [==============================] - 0s 242us/step - loss: 1.0464 - acc: 0.4267
Epoch 159/200
150/150 [==============================] - 0s 226us/step - loss: 0.9996 - acc: 0.4200
Epoch 160/200
150/150 [==============================] - 0s 299us/step - loss: 1.0275 - acc: 0.3933
Epoch 161/200
150/150 [==============================] - 0s 261us/step - loss: 1.0451 - acc: 0.3933
Epoch 162/200
150/150 [==============================] - 0s 260us/step - loss: 1.0345 - acc: 0.3800
Epoch 163/200
150/150 [==============================] - 0s 292us/step - loss: 0.9972 - acc: 0.3733
Epoch 164/200
150/150 [==============================] - 0s 237us/step - loss: 1.0079 - acc: 0.4400
Epoch 165/200
150/150 [==============================] - 0s 239us/step - loss: 0.9920 - acc: 0.3600
Epoch 166/200


ValueError: could not broadcast input array from shape (4,3) into shape (3,3)

In [150]:
inputX = []
inputY = []

#initialize
for i in range(np.shape(matrix)[0]):
    tempX = []
    for j in range(3):
        tempX2 = []
        for k in range(3):
            tempX2.append(matrix[i][j][k])
        tempX.append(tempX2)
    inputX.append(tempX)
    inputY.append(matrix[i][3][0])

print(inputY)

for j in range(3, 4): #to predict jth result
    if j!=3:
        for i in range(np.shape(matrix)[0]):
            #merged = []
            #for l in range(j-1):
                #merged.append( list(inputX[0][l]) )
            #print(merged)
            
            #merged.append([inputY[i], inputX[i][j-2][1], inputX[i][j-2][2]])
            inputX[i].append([inputY[i], inputX[i][j-2][1], inputX[i][j-2][2]])
            
    inputX = np.asarray(inputX)          
    model = Sequential()
    model.add(LSTM(units=30, return_sequences=True, input_shape=(inputX.shape[1], 3)))
    model.add(LSTM(units=30, return_sequences=True))
    model.add(LSTM(units=30))
    model.add(Dense(units=1))
    print(model.summary())
    
    print('MODEL IS TRAINING TO PREDICT THE ' + str(j+1) + 'TH RANK')
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    model.fit(inputX, inputY, epochs=1, batch_size=32)
    
    inputY = model.predict(inputX)
    inputY = inputY.tolist()
    inputX = inputX.tolist()

[3.0, 5.0, 13.0, 4.0, 5.0, 3.0, 12.0, 3.0, 7.0, 8.0, 8.0, 2.0, 7.0, 13.0, 3.0, 1.0, 1.0, 5.0, 2.0, 15.0, 5.0, 6.0, 8.0, 6.0, 3.0, 4.0, 1.0, 12.0, 8.0, 12.0, 9.0, 6.0, 4.0, 3.0, 3.0, 7.0, 2.0, 2.0, 8.0, 7.0, 6.0, 1.0, 5.0, 15.0, 5.0, 9.0, 5.0, 11.0, 2.0, 7.0, 6.0, 3.0, 2.0, 7.0, 4.0, 2.0, 5.0, 1.0, 2.0, 3.0, 6.0, 2.0, 5.0, 3.0, 10.0, 16.0, 4.0, 10.0, 2.0, 5.0, 8.0, 3.0, 6.0, 3.0, 4.0, 6.0, 3.0, 2.0, 6.0, 1.0, 4.0, 5.0, 4.0, 3.0, 19.0, 3.0, 14.0, 5.0, 9.0, 3.0, 5.0, 5.0, 5.0, 5.0, 1.0, 2.0, 6.0, 4.0, 16.0, 1.0, 10.0, 3.0, 1.0, 2.0, 5.0, 9.0, 6.0, 6.0, 4.0, 1.0, 6.0, 18.0, 4.0, 6.0, 4.0, 7.0, 5.0, 3.0, 2.0, 3.0, 3.0, 18.0, 12.0, 4.0, 2.0, 10.0, 4.0, 5.0, 3.0, 14.0, 1.0, 6.0, 4.0, 10.0, 4.0, 12.0, 4.0, 1.0, 3.0, 19.0, 15.0, 5.0, 9.0, 4.0, 1.0, 5.0, 9.0, 9.0, 6.0, 4.0]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_64 (LSTM)               (None, 3, 30)             4080      
___________________________

In [155]:
inputY_flat = [item for sublist in inputY for item in sublist]
inputY_flat
inputY = inputY_flat

In [156]:
for j in range(4, 5): #to predict jth result
    if j!=3:
        for i in range(np.shape(matrix)[0]):
            #merged = []
            #for l in range(j-1):
                #merged.append( list(inputX[0][l]) )
            #print(merged)
            
            #merged.append([inputY[i], inputX[i][j-2][1], inputX[i][j-2][2]])
            #print(np.asarray(merged))
            print('Adding')
            print([inputY[i], inputX[i][j-2][1], inputX[i][j-2][2]])
            inputX[i].append([inputY[i], inputX[i][j-2][1], inputX[i][j-2][2]])
            print(i)
            
    inputX = np.asarray(inputX)          
    model = Sequential()
    model.add(LSTM(units=30, return_sequences=True, input_shape=(inputX.shape[1], 3)))
    model.add(LSTM(units=30, return_sequences=True))
    model.add(LSTM(units=30))
    model.add(Dense(units=1))
    print(model.summary())
    
    print('MODEL IS TRAINING TO PREDICT THE ' + str(j+1) + 'TH RANK')
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    model.fit(inputX, inputY, epochs=200, batch_size=32)
    
    inputY = model.predict(inputX)

Adding
[0.09096435457468033, 0.05, 45.0]
0
Adding
[0.08481013774871826, 0.3, 0.0]
1
Adding
[0.12099635601043701, 0.3, 0.0]
2
Adding
[0.11354520171880722, 0.35, 94.0]
3
Adding
[0.12233404070138931, 0.35, 94.0]
4
Adding
[0.1008753627538681, 0.3, 21.0]
5
Adding
[0.09923791885375977, 0.2, 0.0]
6
Adding
[0.05559985712170601, 0.45, 0.0]
7
Adding
[0.06577669829130173, 0.45, 0.0]
8
Adding
[0.08260475099086761, 0.25, 18.0]
9
Adding
[0.08483844995498657, 0.5, 28.0]
10
Adding
[0.08551351726055145, 0.65, 0.0]
11
Adding
[0.11372724175453186, 0.65, 0.0]
12
Adding
[0.12439975142478943, 0.65, 0.0]
13
Adding
[0.06520895659923553, 0.25, 42.0]
14
Adding
[0.08134234696626663, 0.25, 43.0]
15
Adding
[0.08001936972141266, 0.3, 76.0]
16
Adding
[0.08043723553419113, 0.3, 68.0]
17
Adding
[0.06399943679571152, 0.35, 124.0]
18
Adding
[0.1217225044965744, 0.4, 23.0]
19
Adding
[0.07506795972585678, 0.25, 100.0]
20
Adding
[0.08009247481822968, 0.1, 74.0]
21
Adding
[0.11263468116521835, 0.25, 0.0]
22
Adding
[0.082029

150/150 [==============================] - 0s 417us/step - loss: 5.5962e-05 - acc: 0.0000e+00
Epoch 9/200
150/150 [==============================] - 0s 489us/step - loss: 4.3450e-05 - acc: 0.0000e+00
Epoch 10/200
150/150 [==============================] - 0s 372us/step - loss: 3.4996e-05 - acc: 0.0000e+00
Epoch 11/200
150/150 [==============================] - 0s 429us/step - loss: 2.9386e-05 - acc: 0.0000e+00
Epoch 12/200
150/150 [==============================] - 0s 445us/step - loss: 2.6588e-05 - acc: 0.0000e+00
Epoch 13/200
150/150 [==============================] - 0s 454us/step - loss: 2.0758e-05 - acc: 0.0000e+00
Epoch 14/200
150/150 [==============================] - 0s 492us/step - loss: 1.8819e-05 - acc: 0.0000e+00
Epoch 15/200
150/150 [==============================] - 0s 485us/step - loss: 1.5685e-05 - acc: 0.0000e+00
Epoch 16/200
150/150 [==============================] - 0s 485us/step - loss: 1.4342e-05 - acc: 0.0000e+00
Epoch 17/200
150/150 [=============================

150/150 [==============================] - 0s 366us/step - loss: 1.3125e-06 - acc: 0.0000e+00
Epoch 85/200
150/150 [==============================] - 0s 419us/step - loss: 1.2832e-06 - acc: 0.0000e+00
Epoch 86/200
150/150 [==============================] - 0s 412us/step - loss: 1.2165e-06 - acc: 0.0000e+00
Epoch 87/200
150/150 [==============================] - 0s 432us/step - loss: 1.2424e-06 - acc: 0.0000e+00
Epoch 88/200
150/150 [==============================] - 0s 379us/step - loss: 1.2604e-06 - acc: 0.0000e+00
Epoch 89/200
150/150 [==============================] - 0s 335us/step - loss: 1.3319e-06 - acc: 0.0000e+00
Epoch 90/200
150/150 [==============================] - 0s 360us/step - loss: 1.1735e-06 - acc: 0.0000e+00
Epoch 91/200
150/150 [==============================] - 0s 357us/step - loss: 1.1803e-06 - acc: 0.0000e+00
Epoch 92/200
150/150 [==============================] - 0s 338us/step - loss: 1.1546e-06 - acc: 0.0000e+00
Epoch 93/200
150/150 [============================

150/150 [==============================] - 0s 532us/step - loss: 6.4934e-07 - acc: 0.0000e+00
Epoch 161/200
150/150 [==============================] - 0s 555us/step - loss: 5.2608e-07 - acc: 0.0000e+00
Epoch 162/200
150/150 [==============================] - 0s 539us/step - loss: 4.3632e-07 - acc: 0.0000e+00
Epoch 163/200
150/150 [==============================] - 0s 505us/step - loss: 5.0747e-07 - acc: 0.0000e+00
Epoch 164/200
150/150 [==============================] - 0s 505us/step - loss: 4.2310e-07 - acc: 0.0000e+00
Epoch 165/200
150/150 [==============================] - 0s 591us/step - loss: 4.0140e-07 - acc: 0.0000e+00
Epoch 166/200
150/150 [==============================] - 0s 658us/step - loss: 3.6604e-07 - acc: 0.0000e+00
Epoch 167/200
150/150 [==============================] - 0s 318us/step - loss: 3.6004e-07 - acc: 0.0000e+00
Epoch 168/200
150/150 [==============================] - 0s 326us/step - loss: 3.7774e-07 - acc: 0.0000e+00
Epoch 169/200
150/150 [===================